In [1]:
import numpy as np
import pandas as pd
import transformers
from transformers import AutoModel, BertTokenizerFast, BertConfig, BertModel
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.autograd

import torch.nn.functional as F
import torch
from torch import Tensor
from torch.nn.parameter import Parameter,   UninitializedParameter
from torch.nn import functional as F

In [2]:
def func_collecting_tensors(step, tensor1, tensor2=None):
    '''собирает в тензор     '''
    if step == 0:
        return tensor1.unsqueeze(0)
    else:
        return torch.concatenate((tensor1, tensor2),0)
    


def Tucker_Decomposition(tensor):
    n1, n2, n3 = tensor.shape
    u1, _, _ = torch.svd(torch.reshape(tensor, (n1, -1)))
    u2, _, _ = torch.svd(torch.reshape(torch.permute(tensor, [1, 2, 0]), (n2, -1)))
    u3, _, _ = torch.svd(torch.reshape(torch.permute(tensor, [2, 0, 1]), (n3, -1)))
    return u1, u2, u3


def get_tucker_tensors(dict_layers):
    '''делает словарь где ключом будет слой, а значением будет тензор'''        
    dict_tensor = dict(zip(range(12), [[]]*12))
    for key in dict_layers.keys():
        dict_tensor[key].append(torch.cat(dict_layers[key], 2 ))
    return dict_tensor

In [3]:
class MyLinearFunction(torch.autograd.Function):

        # Note that forward, setseup_context, and backward are @staticmethods
    @staticmethod
    def forward(ctx,input, weight, bias, grads, treshold):
        treshold = treshold
        if ((len(grads) <30 )): 

            ctx.save_for_backward(input, weight, bias)
        else:
            u1, U, VT = Tucker_Decomposition(torch.cat(MyLayer.grads)) ## <- ???
            ctx.save_for_backward(input,weight, bias, U, VT)
        ctx.size = input.shape[0]

        return  input @ weight.T  + bias
    


    @staticmethod
    def backward(ctx, grad_output):

        if len(ctx.saved_tensors) == 3:
#             print('before 30')
            input,weight, bias = ctx.saved_tensors
            grad_input = grad_weight = grad_bias = None
            if ctx.needs_input_grad[0]:
                grad_input = grad_output @ weight
            if ctx.needs_input_grad[1]:
                grad_weight =  grad_output.T @input
  
            if bias is not None and ctx.needs_input_grad[2]:
                grad_bias = grad_output
            
        elif len(ctx.saved_tensors) == 5:

            input, weight, bias, U, VT = ctx.saved_tensors
            print(U)
            print(VT)
            grad_input = grad_weight = grad_bias = None
            if ctx.needs_input_grad[0]:
                grad_input = grad_output @ weight
            if ctx.needs_input_grad[1]:
                grad_weight = grad_output.T @input 
                grad_weight = U @  grad_weight @ VT
                grad_weight = torch.where(torch.abs(grad_weight) >= treshold, grad_weight, 0)
            if bias is not None and ctx.needs_input_grad[2]:
                grad_bias = grad_output
            
        return grad_input, grad_weight, grad_bias, None, None
        
        


In [4]:
class MyLinear(torch.nn.Module):
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.grads = []
        self.treshold = 1e-3
    def forward(self, x):
            return MyLinearFunction.apply(x, self.weight, self.bias, self.grads, self.treshold)
            

In [5]:
x= torch.ones((1,10))
a = torch.nn.Parameter(torch.randn((10,10)))
b = torch.nn.Parameter(torch.randn((10,10)))
c = torch.nn.Parameter(torch.randn((1, 10)))
u = torch.randn((10,10)) 
vt = torch.randn((10,10))
treshold = 1e-3

In [6]:
MyLayer = MyLinear(10, 10)
MyLayer.weight = nn.Parameter(a, requires_grad = True)
MyLayer.bias = nn.Parameter(c, requires_grad = True)

In [7]:
gt_layer = nn.Linear(10,10)
gt_layer.weight = torch.nn.Parameter(a)
gt_layer.bias = torch.nn.Parameter(c)
optimizer_gt = torch.optim.Adam(gt_layer.parameters())
criterion_gt = nn.MSELoss()

In [8]:
gt = torch.nn.Parameter(torch.randn((1, 10)))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(MyLayer.parameters())

In [9]:
import numpy as np

In [10]:
for i in range(30):

    optimizer.zero_grad()
    optimizer_gt.zero_grad()
    
    output = MyLayer(x)
    w = gt_layer(x)
    assert np.allclose(output.detach().numpy(), w.detach().numpy())
    
    loss = criterion(output, gt)
    loss_gt = criterion_gt(w, gt)
    
    loss.backward()
    loss_gt.backward()
    
#     print(i, gt_layer.weight.detach().numpy())
#     print(MyLayer.weight.grad.detach().numpy())
    assert np.allclose(MyLayer.weight.grad.detach().numpy(), gt_layer.weight.grad.detach().numpy())
    
    MyLayer.grads.append(MyLayer.weight.grad.unsqueeze(0))

    optimizer.step()
    optimizer_gt.step()

In [11]:
optimizer.zero_grad()
optimizer_gt.zero_grad()

output = MyLayer(x)
w = gt_layer(x)
assert np.allclose(output.detach().numpy(), w.detach().numpy())

loss = criterion(output, gt)
loss_gt = criterion_gt(w, gt)

loss.backward()
loss_gt.backward()

#     print(i, gt_layer.weight.detach().numpy())
#     print(MyLayer.weight.grad.detach().numpy())
assert np.allclose(MyLayer.weight.grad.detach().numpy(), gt_layer.weight.grad.detach().numpy())

MyLayer.grads.append(MyLayer.weight.grad.unsqueeze(0))

optimizer.step()
optimizer_gt.step()

tensor([[-0.3243, -0.0407,  0.2942, -0.4176,  0.4783,  0.0510, -0.2926, -0.5308,
         -0.1217,  0.1371],
        [-0.1233, -0.5508, -0.5759, -0.4176, -0.0267,  0.0794,  0.1278, -0.0800,
          0.3484, -0.1556],
        [ 0.1783,  0.4147, -0.2516, -0.3676,  0.0686,  0.3737, -0.0621,  0.0988,
         -0.4063, -0.5242],
        [ 0.4451, -0.2728,  0.3231, -0.4245, -0.4440, -0.1093, -0.4623,  0.1276,
         -0.0368,  0.0494],
        [ 0.1406,  0.5085,  0.2045, -0.2704, -0.0394, -0.0872,  0.1738, -0.1405,
          0.7273, -0.1365],
        [-0.2157, -0.3197,  0.4327,  0.2357,  0.0947,  0.0287, -0.0882,  0.1218,
          0.1445, -0.7451],
        [-0.3949,  0.1425, -0.0346, -0.3353,  0.0597, -0.6869,  0.0559,  0.4530,
         -0.1533, -0.0541],
        [-0.3477,  0.0199,  0.1977, -0.1675,  0.0142,  0.5893, -0.0297,  0.5764,
          0.1838,  0.3118],
        [-0.4397,  0.2589, -0.2905,  0.2014, -0.4200,  0.0061, -0.6235, -0.1550,
          0.1453, -0.0652],
        [ 0.3366,  

AssertionError: 

In [12]:
U_ = torch.tensor([[-0.3243, -0.0407,  0.2942, -0.4176,  0.4783,  0.0510, -0.2926, -0.5308,
         -0.1217,  0.1371],
        [-0.1233, -0.5508, -0.5759, -0.4176, -0.0267,  0.0794,  0.1278, -0.0800,
          0.3484, -0.1556],
        [ 0.1783,  0.4147, -0.2516, -0.3676,  0.0686,  0.3737, -0.0621,  0.0988,
         -0.4063, -0.5242],
        [ 0.4451, -0.2728,  0.3231, -0.4245, -0.4440, -0.1093, -0.4623,  0.1276,
         -0.0368,  0.0494],
        [ 0.1406,  0.5085,  0.2045, -0.2704, -0.0394, -0.0872,  0.1738, -0.1405,
          0.7273, -0.1365],
        [-0.2157, -0.3197,  0.4327,  0.2357,  0.0947,  0.0287, -0.0882,  0.1218,
          0.1445, -0.7451],
        [-0.3949,  0.1425, -0.0346, -0.3353,  0.0597, -0.6869,  0.0559,  0.4530,
         -0.1533, -0.0541],
        [-0.3477,  0.0199,  0.1977, -0.1675,  0.0142,  0.5893, -0.0297,  0.5764,
          0.1838,  0.3118],
        [-0.4397,  0.2589, -0.2905,  0.2014, -0.4200,  0.0061, -0.6235, -0.1550,
          0.1453, -0.0652],
        [ 0.3366,  0.0088, -0.2456,  0.1616,  0.6148, -0.1096, -0.4997,  0.2995,
          0.2629,  0.0402]])
VT_= torch.tensor([[-3.1623e-01,  9.4868e-01,  1.7652e-07,  3.9227e-08,  9.8067e-09,
          0.0000e+00, -9.8067e-09, -1.9613e-08,  2.4517e-08, -1.9613e-08],
        [-3.1623e-01, -1.0541e-01,  9.4281e-01,  2.9293e-08,  2.2224e-08,
          1.9868e-08, -2.3561e-09, -2.4580e-08, -1.3978e-08, -4.7122e-09],
        [-3.1623e-01, -1.0541e-01, -1.1785e-01,  9.3541e-01,  2.7961e-07,
         -1.5202e-08,  1.7813e-08, -2.9849e-08, -2.9331e-08,  3.5627e-08],
        [-3.1623e-01, -1.0541e-01, -1.1785e-01, -1.3363e-01,  9.2582e-01,
         -8.2434e-08,  2.3446e-08,  2.6119e-08,  3.4087e-08,  2.6358e-08],
        [-3.1623e-01, -1.0541e-01, -1.1785e-01, -1.3363e-01, -1.5430e-01,
          9.1287e-01,  7.1498e-08,  1.7301e-08,  2.0023e-08,  3.2441e-08],
        [-3.1623e-01, -1.0541e-01, -1.1785e-01, -1.3363e-01, -1.5430e-01,
         -1.8257e-01,  8.9443e-01,  2.4000e-09,  8.8474e-09, -7.0738e-05],
        [-3.1623e-01, -1.0541e-01, -1.1785e-01, -1.3363e-01, -1.5430e-01,
         -1.8257e-01, -2.2354e-01, -4.2301e-08, -2.4679e-08,  8.6604e-01],
        [-3.1623e-01, -1.0541e-01, -1.1785e-01, -1.3363e-01, -1.5430e-01,
         -1.8257e-01, -2.2363e-01, -5.7735e-01, -5.7735e-01, -2.8866e-01],
        [-3.1623e-01, -1.0541e-01, -1.1785e-01, -1.3363e-01, -1.5430e-01,
         -1.8257e-01, -2.2363e-01, -2.1132e-01,  7.8868e-01, -2.8866e-01],
        [-3.1623e-01, -1.0541e-01, -1.1785e-01, -1.3363e-01, -1.5430e-01,
         -1.8257e-01, -2.2363e-01,  7.8868e-01, -2.1132e-01, -2.8866e-01]])

In [13]:
assert np.allclose(MyLayer.weight.grad.detach().numpy(), U_.detach().numpy()@ gt_layer.weight.grad.detach().numpy() @ VT_.detach().numpy())

AssertionError: 

In [15]:
gradd =  U_.detach().numpy()@ gt_layer.weight.grad.detach().numpy() @ VT_.detach().numpy()


In [16]:
gradd = np.where(np.abs(gradd) >= 1e-3, gradd, 0)

In [17]:
assert np.allclose(MyLayer.weight.grad.detach().numpy(),gradd)

AssertionError: 

In [19]:
MyLayer.weight.grad.detach().numpy()

array([[-3.2636096 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 3.4004018 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 2.9717562 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.88249606,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 4.409864  ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.26655453,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.20412314,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        

In [18]:
gradd

array([[-3.2639494 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 3.400247  ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 2.9717925 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.88257873,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 4.409724  ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.2671327 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.20370713,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        

In [8]:
for i in range(30):
    optimizer.zero_grad()
    output = MyLayer(x)
    loss = criterion(output, gt)
  
    loss.backward()
    MyLayer.grads.append(MyLayer.weight.grad.unsqueeze(0))

    optimizer.step()

In [9]:
optimizer.zero_grad()
output = MyLayer(x)
loss = criterion(output, gt)

loss.backward()

optimizer.step()